Dataframes Needed:
- Live games (Teams playing, scores, results)
- Box scores (Player stats)
- Lead tracker

Home Page
- Live game scores, Youtube videos of highlights

Game Page
- Box scores
- Matchup stats (lead tracker, team %s)
- Youtube highlight

Visualizations
- [Blank]

In [1]:
import nbapy
import datetime
import warnings
import pandas as pd
import dateutil.parser
from nbapy import game
from nbapy.scoreboard import Scoreboard

pd.set_option('max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
test_gameid = '0021901318'
test_date = '08-13-2020'

In [3]:
import constants

### Get Games Function

In [4]:
def get_games(datestring):
    """
    Input:
        datestring (str) - date of NBA games to grab
    
    Output:
        games (pandas df) - dataframe of NBA games, results, and teams playing
        
    """
    date = dateutil.parser.parse(datestring)
    scoreboard = Scoreboard(date.month, date.day, date.year)
    
    # Grab today's game information
    game_header = scoreboard.game_header()
    line_score = scoreboard.line_score()
    
    # Simplify columns
    games = game_header[['GAME_SEQUENCE', 'GAME_ID', 'GAME_STATUS_TEXT', 
                         'LIVE_PERIOD', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID',
                         'NATL_TV_BROADCASTER_ABBREVIATION']]
    score = line_score[['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PTS']]
    
    # Dictionary of game values
    team_info = score.set_index('TEAM_ID')['TEAM_ABBREVIATION'].to_dict()
    team_pts = score.set_index('TEAM_ID')['PTS'].to_dict()
    
    # Map to games info
    games['HOME_TEAM_ABBREVIATION'] = games.HOME_TEAM_ID.map(team_info)
    games['VISITOR_TEAM_ABBREVIATION'] = games.VISITOR_TEAM_ID.map(team_info)
    games['HOME_TEAM_PTS'] = games.HOME_TEAM_ID.map(team_pts)
    games['VISITOR_TEAM_PTS'] = games.VISITOR_TEAM_ID.map(team_pts)
    
    # Create a winner column for web layout
    winners = []
    for i, value in games.iterrows():
        if (value["HOME_TEAM_PTS"] > value["VISITOR_TEAM_PTS"]):
            winners.append(value["HOME_TEAM_ABBREVIATION"])
        elif (value["HOME_TEAM_PTS"] < value["VISITOR_TEAM_PTS"]):
            winners.append(value["VISITOR_TEAM_ABBREVIATION"])
        else:
            winners.append(None)
    
    games['WINNER'] = winners
    
    return games

games = get_games(test_date)
games

,GAME_SEQUENCE,GAME_ID,GAME_STATUS_TEXT,LIVE_PERIOD,HOME_TEAM_ID,VISITOR_TEAM_ID,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_PTS,VISITOR_TEAM_PTS,WINNER
0,1,0021901308,Final,4,1610612738,1610612764,None,BOS,WAS,90,96,WAS
1,2,0021901310,Final,4,1610612747,1610612758,None,LAL,SAC,122,136,SAC
2,3,0021901311,Final,4,1610612763,1610612749,None,MEM,MIL,119,106,MEM
3,4,0021901313,Final,4,1610612756,1610612742,TNT,PHX,DAL,128,102,PHX
4,5,0021901314,Final,4,1610612762,1610612759,TNT,UTA,SAS,118,112,UTA
5,6,0021901309,Final,4,1610612751,1610612757,TNT,BKN,POR,133,134,POR
6,7,0021901312,Final,4,1610612753,1610612740,None,ORL,NOP,133,127,ORL


### Get Live Games

Columns Needed
- GAME_STATUS_TEXT
- HOME_TEAM_ABBREVIATION
- VISITOR_TEAM_ABBREVIATION
- HOME_TEAM_PTS
- VISITOR_TEAM_PTS

In [5]:
def get_livegames():
    """
    Input:
        None
        
    Output:
        live_games (pandas df) - Dataframe showing results for today's NBA games
    """
    today = datetime.datetime.now()
    today_string = today.strftime("%m-%d-%Y")
    live_games = get_games(today_string)
    live_games = live_games.fillna(0)
    live_games[["HOME_TEAM_PTS", "VISITOR_TEAM_PTS"]] = live_games[["HOME_TEAM_PTS", "VISITOR_TEAM_PTS"]].astype(int)
    return live_games

live_games = get_livegames()
live_games

,GAME_SEQUENCE,GAME_ID,GAME_STATUS_TEXT,LIVE_PERIOD,HOME_TEAM_ID,VISITOR_TEAM_ID,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_PTS,VISITOR_TEAM_PTS,WINNER
0,1,0041900113,Start of 4th Qtr,4,1610612751,1610612761,NBA TV,BKN,TOR,68,84,TOR
1,2,0041900163,4:00 pm ET,0,1610612762,1610612743,TNT,UTA,DEN,0,0,0
2,3,0041900123,6:30 pm ET,0,1610612755,1610612738,TNT,PHI,BOS,0,0,0
3,4,0041900153,9:00 pm ET,0,1610612742,1610612746,TNT,DAL,LAC,0,0,0


### Get Box Scores

In [81]:
def get_boxscore(game_id):
    """
    Input:
        game_id (str) - Game ID to pull box scores
    
    Output:
        box_score (pandas df) - Dataframe containing the box score results of Game ID
        
    """
    boxscore = game.BoxScore(game_id).players_stats()
    
    # Data cleaning
    boxscore = boxscore.fillna(0)
    float_scores = boxscore.select_dtypes('float')
    headers = boxscore.columns.values
    
    # Keep PCT as floats
    pct_df = float_scores.filter(regex='PCT')
    pct_cols = pct_df.columns.values
    boxscore.drop(pct_cols, axis=1, inplace=True)
    boxscore = pd.concat([boxscore, pct_df], axis=1)
    
    # Convert stats to ints
    int_df = float_scores.astype(int).drop(pct_cols, axis=1)
    int_cols = int_df.columns.values
    boxscore.drop(int_cols, axis=1, inplace=True)
    boxscore = pd.concat([boxscore, int_df], axis=1)
    
    # Keep original order of headers
    boxscore = boxscore[headers]
    team1, team2 = boxscore.groupby("TEAM_ID")
    return team1[1], team2[1]

team1, team2 = get_boxscore(test_gameid)
team1

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0021901318,1610612743,DEN,Denver,203924,Jerami Grant,F,,18:18,1,6,0.167,0,3,0.000,0,0,0.000,1,1,2,1,0,0,0,0,2,2
1,0021901318,1610612743,DEN,Denver,200794,Paul Millsap,F,,15:30,4,8,0.500,2,3,0.667,0,0,0.000,1,2,3,2,2,1,1,2,10,-4
2,0021901318,1610612743,DEN,Denver,203999,Nikola Jokic,C,,10:12,1,1,1.000,0,0,0.000,0,0,0.000,0,1,1,2,0,0,1,0,2,-6
3,0021901318,1610612743,DEN,Denver,1628470,Torrey Craig,G,,14:58,1,6,0.167,1,3,0.333,0,0,0.000,1,3,4,3,0,0,1,0,3,-6
4,0021901318,1610612743,DEN,Denver,1627750,Jamal Murray,G,,10:12,4,5,0.800,3,3,1.000,0,0,0.000,0,0,0,1,1,0,0,2,11,-6
5,0021901318,1610612743,DEN,Denver,1628420,Monte Morris,,,29:51,4,14,0.286,2,7,0.286,6,6,1.000,0,6,6,4,0,1,2,0,16,-7
6,0021901318,1610612743,DEN,Denver,1628408,PJ Dozier,,,30:48,6,9,0.667,5,6,0.833,3,3,1.000,0,5,5,8,2,0,1,0,20,-5
7,0021901318,1610612743,DEN,Denver,1628966,Keita Bates-Diop,,,24:21,3,11,0.273,1,4,0.250,4,4,1.000,2,4,6,0,1,2,2,3,11,-1
8,0021901318,1610612743,DEN,Denver,203486,Mason Plumlee,,,19:42,4,6,0.667,0,0,0.000,2,3,0.667,1,2,3,2,0,4,0,2,10,-3
9,0021901318,1610612743,DEN,Denver,203584,Troy Daniels,,,28:00,4,11,0.364,2,8,0.250,0,0,0.000,0,3,3,1,1,0,3,3,10,-2


In [120]:
team2

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
13,0021901318,1610612761,TOR,Toronto,1626181,Norman Powell,F,,22:48,6,9,0.667,3,5,0.600,0,0,0.000,1,5,6,2,1,1,3,2,15,-3
14,0021901318,1610612761,TOR,Toronto,1626178,Rondae Hollis-Jefferson,F,,22:18,3,9,0.333,0,0,0.000,0,1,0.000,2,4,6,3,0,1,1,1,6,11
15,0021901318,1610612761,TOR,Toronto,201188,Marc Gasol,C,,16:43,1,3,0.333,0,1,0.000,0,0,0.000,0,3,3,5,1,1,2,2,2,-2
16,0021901318,1610612761,TOR,Toronto,1629744,Matt Thomas,G,,28:18,4,6,0.667,2,3,0.667,2,2,1.000,1,2,3,2,1,0,2,4,12,7
17,0021901318,1610612761,TOR,Toronto,1629056,Terence Davis,G,,34:02,5,15,0.333,2,6,0.333,2,2,1.000,1,5,6,3,2,0,3,2,14,6
18,0021901318,1610612761,TOR,Toronto,1628449,Chris Boucher,,,21:30,3,8,0.375,1,3,0.333,2,3,0.667,4,5,9,0,0,1,1,2,9,5
19,0021901318,1610612761,TOR,Toronto,1628778,Paul Watson,,,27:28,8,13,0.615,4,6,0.667,2,3,0.667,0,6,6,2,1,0,1,2,22,8
20,0021901318,1610612761,TOR,Toronto,1626169,Stanley Johnson,,,28:34,9,16,0.563,4,8,0.500,1,3,0.333,0,4,4,6,1,1,5,5,23,5
21,0021901318,1610612761,TOR,Toronto,1626259,Malcolm Miller,,,25:37,4,5,0.800,2,3,0.667,0,0,0.000,1,0,1,3,1,0,0,4,10,3
22,0021901318,1610612761,TOR,Toronto,1629608,Dewan Hernandez,,,12:42,2,6,0.333,0,0,0.000,0,0,0.000,3,4,7,1,0,0,1,0,4,0


### Get Team Stats

In [9]:
def get_teamstats(game_id):
    """
    Input:
        game_id (str) - Game ID to pull team stats
        
    Output:
        team_stats (pandas df) - Dataframe showing the teams percentages
    
    """
    team_stats = game.BoxScore(game_id).team_stats()
    return team_stats

team_stats = get_teamstats(test_gameid)
team_stats

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0021901318,1610612761,Raptors,TOR,Toronto,240:00,45,90,0.500,18,35,0.514,9,14,0.643,13,38,51,27,8,5,19,24,117,8.0
1,0021901318,1610612743,Nuggets,DEN,Denver,240:00,36,87,0.414,16,38,0.421,21,23,0.913,9,32,41,27,9,11,15,15,109,-8.0


In [121]:
team_stats.T

,0,1
GAME_ID,0021901318,0021901318
TEAM_ID,1610612761,1610612743
TEAM_NAME,Raptors,Nuggets
TEAM_ABBREVIATION,TOR,DEN
TEAM_CITY,Toronto,Denver
MIN,240:00,240:00
FGM,45,36
FGA,90,87
FG_PCT,0.5,0.414
FG3M,18,16


### Get Lead Tracker

In [10]:
def get_linescore(game_id):
    """
    Input:
        game_id (str) - Game ID to pull line score
    
    Ouput:
        line_score (pandas df) - Dataframe showing the points by quarter
    
    """
    line_score = game.Info(game_id).line_score()
    return line_score

line_score = get_linescore(test_gameid)
line_score

,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY_NAME,TEAM_NICKNAME,TEAM_WINS_LOSSES,PTS_QTR1,PTS_QTR2,PTS_QTR3,PTS_QTR4,PTS_OT1,PTS_OT2,PTS_OT3,PTS_OT4,PTS_OT5,PTS_OT6,PTS_OT7,PTS_OT8,PTS_OT9,PTS_OT10,PTS
0,2020-08-14T00:00:00,1,0021901318,1610612743,DEN,Denver,Nuggets,46-27,23,27,26,33,0,0,0,0,0,0,0,0,0,0,109
1,2020-08-14T00:00:00,1,0021901318,1610612761,TOR,Toronto,Raptors,53-19,31,27,27,32,0,0,0,0,0,0,0,0,0,0,117


### Get Standings

In [11]:
scoreboard = Scoreboard()
east_standings = scoreboard.east_conf_standings_by_day()
west_standings = scoreboard.west_conf_standings_by_day()

In [12]:
west_standings

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD
0,1610612747,00,22019,08/21/2020,West,L.A. Lakers,71,52,19,0.732,25-10,27-9
1,1610612746,00,22019,08/21/2020,West,LA Clippers,72,49,23,0.681,27-9,22-14
2,1610612743,00,22019,08/21/2020,West,Denver,73,46,27,0.630,26-11,20-16
3,1610612745,00,22019,08/21/2020,West,Houston,72,44,28,0.611,24-12,20-16
4,1610612760,00,22019,08/21/2020,West,Oklahoma City,72,44,28,0.611,23-14,21-14
5,1610612762,00,22019,08/21/2020,West,Utah,72,44,28,0.611,23-12,21-16
6,1610612742,00,22019,08/21/2020,West,Dallas,75,43,32,0.573,20-18,23-14
7,1610612757,00,22019,08/21/2020,West,Portland,74,35,39,0.473,21-15,14-24
8,1610612763,00,22019,08/21/2020,West,Memphis,73,34,39,0.466,20-17,14-22
9,1610612756,00,22019,08/21/2020,West,Phoenix,73,34,39,0.466,17-22,17-17


### Workspace

In [14]:
test_list = ['08/20/2020']
test_list

['08/20/2020']

In [15]:
test_list[0]

'08/20/2020'